In [1]:
import pandas as pd

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

    если источник traffic_source равен yandex или google, то в source_type ставится organic
    для источников paid и email из России - ставим ad
    для источников paid и email не из России - ставим other
    все остальные варианты берем из traffic_source без изменений

In [2]:
traffic = pd.read_csv('visit_log.csv',sep = ';')
traffic['source_type'] = traffic['traffic_source'] #создали клон, для тех, что без изменений
traffic.loc[traffic['traffic_source'].isin(['yandex','google']),'source_type'] = 'organic' #заменили гугл и яндекс
traffic.loc[(traffic.region.eq('Russia') &  
             traffic.traffic_source.eq('paid')),'source_type'] = 'ad' #заменили paid из России на ad
traffic.loc[(~traffic.region.eq('Russia') &  
             traffic.traffic_source.eq('paid')),'source_type'] = 'other'
traffic.source_type.value_counts()

direct     7013
organic    5407
email      3875
ad         1581
other      1062
Name: source_type, dtype: int64

## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

* Прочитайте содержимое файла с датафрейм
* Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [3]:
urls = pd.read_csv('URLs.txt')
urls.head(15)
urls[urls.url.str.contains('\d{8}[-]')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [4]:
ratings = pd.read_csv('ratings.csv')

ratings_max = ratings.groupby('userId').max().reset_index()
ratings_min = ratings.groupby('userId').min().reset_index()
ratings_pt = ratings.pivot_table(index = 'userId', 
                                 columns = 'rating', 
                                 values = 'timestamp', 
                                 margins = 'true', 
                                 aggfunc = 'count'
                                )
filteredratings = ratings_pt.query('All > 100').reset_index()
filtered_rat_max = filteredratings.join(ratings_max['timestamp'], 
                                        on = 'userId', how = 'left').rename(columns = {'timestamp':'Timestamp_max'})
filtered_rat_min_max = filtered_rat_max.join(ratings_min['timestamp'], 
                                             on = 'userId', how = 'left').rename(columns = {'timestamp':'Timestamp_min'})
filtered_rat_min_max['time_of_life'] = filtered_rat_min_max['Timestamp_max'] - filtered_rat_min_max['Timestamp_min']
filtered_rat_min_max[['userId','time_of_life']].time_of_life.mean()

19977971.459016394

## Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [5]:
# #Дана статистика услуг перевозок клиентов компании по типам:

#     rzd - железнодорожные перевозки
#     auto - автомобильные перевозки
#     air - воздушные перевозки
#     client_base - адреса клиентов
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
clients_without_address = rzd.merge(auto, how = 'outer').merge(air, how = 'outer')
clients_with_address = clients_without_address.merge(client_base, how = 'outer')
clients_with_address, clients_without_address

(   client_id  rzd_revenue  auto_revenue  air_revenue          address
 0        111       1093.0           NaN          NaN  Комсомольская 4
 1        112       2810.0           NaN          NaN   Энтузиастов 8а
 2        113      10283.0       57483.0          NaN  Левобережная 1а
 3        114       5774.0          83.0          NaN          Мира 14
 4        115        981.0         912.0         81.0        ЗЖБИиДК 1
 5        116          NaN        4834.0          4.0    Строителей 18
 6        117          NaN          98.0         13.0  Панфиловская 33
 7        118          NaN           NaN        173.0     Мастеркова 4,
    client_id  rzd_revenue  auto_revenue  air_revenue
 0        111       1093.0           NaN          NaN
 1        112       2810.0           NaN          NaN
 2        113      10283.0       57483.0          NaN
 3        114       5774.0          83.0          NaN
 4        115        981.0         912.0         81.0
 5        116          NaN        48